In [2]:
!pip install scikeras
!pip install tensorflow

  Using cached keras-3.5.0-py3-none-any.whl.metadata (5.8 kB)
Using cached keras-3.5.0-py3-none-any.whl (1.1 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.5.0 which is incompatible.
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires ke

Leer y procesar imágenes para entrenar:

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import tensorflow as tf
import matplotlib as plt
from sklearn.model_selection import KFold
from keras import layers, models
import os
import cv2
from scikeras.wrappers import KerasClassifier

# tamaño de imagen y epocas del entrenamiento
width = 100
height = 100

ruta_train = '/content/drive/My Drive/chest_xray/train/'

train_x = []
train_y = []

# leer y procesar imagenes
for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train + i + '/' + j, cv2.IMREAD_GRAYSCALE)  # leer imagen en escala de grises
        if img is None:
            continue

        resized = cv2.resize(img, (width, height))
        resized = resized / 255.0  # normalizar a [0, 1]

        train_x.append(resized)

        if i == "NORMAL":
            train_y.append([0, 1])
        else:
            train_y.append([1, 0])

x_data = np.array(train_x)
y_data = np.array(train_y)

# añadir una dimensión extra para que coincida con la entrada del modelo (capa Conv2D espera (width, height, 1))
x_data = np.expand_dims(x_data, axis=-1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Definición y entrenamiento del modelo:

In [4]:
def create_model():
    model = tf.keras.Sequential([
        layers.Conv2D(1000, (3, 3), input_shape=(width, height, 1), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True)

accuracy_scores = []

for train_index, test_index in kf.split(x_data):
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    # Crear y entrenar el modelo para cada fold
    model = create_model()
    epochs = 4
    model.fit(x_train, y_train, epochs=epochs, batch_size=32, verbose=0)

    # Evaluar el modelo en el conjunto de prueba
    score = model.evaluate(x_test, y_test, verbose=0)
    accuracy_scores.append(score[1])  # score[1] es la precisión (accuracy)

# Mostrar los resultados de la validación cruzada
print("Cross-Validation accuracy scores: ", accuracy_scores)
print("Mean accuracy: ", np.mean(accuracy_scores))

# Guardar el último modelo entrenado
model.save('aprendePorfavor.keras')

Cross-Validation accuracy scores:  [0.947858452796936, 0.9199255108833313, 0.871268630027771, 0.9328358173370361, 0.9402984976768494]
Mean accuracy:  0.9224373817443847


Cargar y procesar imágenes para evaluar el modelo:

In [6]:
import numpy as np
import tensorflow as tf
from keras import layers, models
from google.colab import drive
import os
import cv2

#tamaño de imagen
width = 100
height = 100

# ruta hacia las imagenes para testear
ruta_test = '/content/drive/My Drive/chest_xray/test/'

test_x = []
test_y = []

# cargar el modelo
model = models.load_model('aprendePorfavor.keras')

# leer y procesar imagenes
for i in os.listdir(ruta_test):
    for j in os.listdir(ruta_test + i):
        img = cv2.imread(ruta_test + i + '/' + j, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        resized = cv2.resize(img, (width, height))
        resized = resized / 255.0

        test_x.append(resized)

        if i == "NORMAL":
            test_y.append([0, 1])
        else:
            test_y.append([1, 0])

x_test = np.array(test_x)
y_test = np.array(test_y)

x_test = np.expand_dims(x_test, axis=-1)

Evaluar el modelo:

In [7]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

20/20 [==============================] - 8s 388ms/step - loss: 0.4296 - accuracy: 0.8173
Test accuracy: 0.817307710647583
